# Исследование надежности заемщиков


**Цель исследования**
    
Выявить, существует ли зависимость между семейным положением и количеством детей на погашение кредита в срок.

**План работы**
1. Выгрузить датафрейм с данными;
2. Посмотреть общую информацию о данных;
3. Избавиться от дубликатов и пропусков, если таковы имеются;
4. Проанализировать данные;
5. Сделать соответствующие выводы.
    
**Входные данные от банка — статистика о платёжеспособности клиентов.**


## Выгрузка

In [ ]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [ ]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

В 2-х столбцах датафрейма есть одинаковое количество пропусков. Также в столбце "days_employed" заметили отрицательные значения, которых не должно быть, поскольку этот столбец означает количество рабочих дней.

## Предобработка данных

### Удаление пропусков

In [ ]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [ ]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

In [ ]:
data['days_employed'] = data['days_employed'].abs()

In [ ]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

In [ ]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [ ]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [ ]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

In [ ]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = data.loc[(data['income_type'] == t), 'days_employed'].median()

In [ ]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

In [ ]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

In [ ]:
data['education'] = data['education'].str.lower()

In [ ]:
data.duplicated().sum()

71

In [ ]:
data = data.drop_duplicates()

### Категоризация данных

In [ ]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [ ]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [ ]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [ ]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [ ]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

**Вывод**

Мы предобработали данные: заменили пропуски в столбцах, удалили аномальные значения, привели данные к общему виду и удалили дубликаты.
Произвели категоризацию для удобства анализа данных и вывели результат в новые столбцы.
</div>

### Анализ данных

#### Есть ли зависимость между количеством детей и возвратом кредита в срок

In [ ]:
data.groupby('children')['debt'].sum()

children
0    1063
1     444
2     194
3      27
4       4
5       0
Name: debt, dtype: int64

**Вывод:** В датафрейме показано, что в семьях присутствует корреляция между количеством детей и долгом. Чем меньше детей в семье, тем чаще люди не возвращали деньги в срок. Но можно ещё сказать, что семей с 3-я и более детьми гораздо реже берут деньги в долг у банков.

In [ ]:
data.pivot_table(index='children', values='debt', aggfunc={'count', 'mean'})

,count,mean
children,,
0,14091.0,0.075438
1,4808.0,0.092346
2,2052.0,0.094542
3,330.0,0.081818
4,41.0,0.097561
5,9.0,0.000000


**Вывод**

Самый большой процент не возвратов денег в срок прослеживается среди людей с 4-я детьми. Это скорее всего связано с маленьким числом заёмщиков в этой категории.

Можно сказать, что чаще всего не выплачивают кредит в срок категории с 2 и 3 детьми, потому что в этих случаях число заёмщиков достаточно значительное.</div>

#### Есть ли зависимость между семейным положением и возвратом кредита в срок

In [ ]:
# Сгруппируем датафрейм по столбцам "семейный статус" и "долг" и применим sum()
data.groupby('family_status')['debt'].sum()

family_status
Не женат / не замужем    273
в разводе                 84
вдовец / вдова            63
гражданский брак         385
женат / замужем          927
Name: debt, dtype: int64

**Вывод:** Женатые / замужние люди почти в 3 раза чаще не возвращали кредит в срок, чем люди, состоящие в гражданском браке. Самыми ответственными заемщиками являются вдовцы / вдовы и разведенные пары.

In [ ]:
data.pivot_table(index='family_status', values='debt', aggfunc={'count', 'mean'})

,count,mean
family_status,,
Не женат / не замужем,2796.0,0.097639
в разводе,1189.0,0.070648
вдовец / вдова,951.0,0.066246
гражданский брак,4134.0,0.093130
женат / замужем,12261.0,0.075606


#### Есть ли зависимость между уровнем дохода и возвратом кредита в срок

In [ ]:
data.groupby('total_income_category')['debt'].sum()

total_income_category
A       2
B     354
C    1353
D      21
E       2
Name: debt, dtype: int64

**Вывод:** Основная масса людей, которые не вернули кредит в срок имеют доход в размере от 50 до 200 тыс. рублей. Так же достаточно большое количество должников наблюдается среди заемщиков с доходов от 30 до 50 тыс. рублей.

In [ ]:
data.pivot_table(index='total_income_category', values='debt', aggfunc={'count', 'mean'})

,count,mean
total_income_category,,
A,25.0,0.080000
B,5014.0,0.070602
C,15921.0,0.084982
D,349.0,0.060172
E,22.0,0.090909


**Вывод**

Заемщики с самым низким доходом чаще не выплачивают кредиты в срок. На это повлияло то, что количество заемщиков в этой категории меньше всего. Затем следуют люди, получающие от 50 до 200 тысяч, их процент не выплат от общего числа составляет 8,4.

#### Как разные цели кредита влияют на его возврат в срок

In [ ]:
data.groupby('purpose_category')['debt'].sum()

purpose_category
операции с автомобилем      400
операции с недвижимостью    780
получение образования       369
проведение свадьбы          183
Name: debt, dtype: int64

**Вывод:** Чаще всего деньги не возвращались в срок, если были взяты для операций с недвижимостью. Меньше всего не возвратов в срок среди операций, связанных с проведением свадьбы.

In [ ]:
data.pivot_table(index='purpose_category', values='debt', aggfunc={'count', 'mean'})

,count,mean
purpose_category,,
операции с автомобилем,4279.0,0.093480
операции с недвижимостью,10751.0,0.072551
получение образования,3988.0,0.092528
проведение свадьбы,2313.0,0.079118


**Вывод**

Заемщики, которые брали кредиты для операций с автомобилем имеют 9,3% не выплат в срок от общего числа. Для образования такой процент составляет 9,2.

#### Возможные причины появления пропусков в исходных данных

*Ответ:* Пропуски в данных свидетельствуют о сбое системы, поскольку отсутствует одинаковое количество ячеек в 2-х столбцах.

## Вывод

**Общий вывод**

За основу анализа данных была взята статистика о платежеспособности клиентов банка.
На этапе преобработки провели следующие операции с данными:
- вывели общую информацию о датафрейме
- заполнили пропуски, удалили аномальные значения и дубликаты
- привели данные к общему виду
- произвели категоризацию по количеству дохода и цели кредита

Отвечая на вопрос есть ли зависимость между семейным положением, количеством детей и выплатой кредита в срок, выяснили, что при наличии 2-х детей люди чаще просрочивают оплату кредита, также не выплачивают в срок не женатые и состоящие в гражданском браке, имеющие доход от 50 до 200 тысяч и те, кто берёт кредит для операций с автомобилем.

**Идеальным заёмщиком** будет человек *без детей*, *женатый*, получающий *от 200 тысяч до 1 миллиона* и целью кредита у которого будет *проведение свадьбы* или *операции с недвижимостью*.